In [2]:
!pip3 install datasets
!pip3 install evaluate
!pip3 install transformers
!pip3 install adapters
!pip3 install accelerate==0.30

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import logging
import os
import random
import sys
import time
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from adapters import AdapterArguments, AdapterTrainer, AutoAdapterModel, setup_adapter_training, AdapterConfig
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

/home/hice1/maleem3/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
task_to_keys = {
    "boolq": ("question", "passage"),
    "cb": ("premise", "hypothesis"),
    "copa": ("premise", "choice1", "choice2"),
    "multirc": ("paragraph", "question"),
    "record": ("passage", "query"),
    "rte": ("premise", "hypothesis"),
    "wic": ("sentence1", "sentence2"),
    "wsc": ("text", None),
}

In [5]:
def filter_unused_args(args):
    filtered_args = []
    for arg in args:
        if not arg.startswith("-f") and not (arg.endswith(".json") or arg.endswith(".py")):
            filtered_args.append(arg)
    return filtered_args
@dataclass
class DataTrainingArguments:
    task_name: Optional[str] = field(
        default='boolq',
        metadata={"help": "The name of the task to train on: " + ", ".join(task_to_keys.keys())},
    )
    dataset_name: Optional[str] = field(
        default='super_glue', metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached preprocessed datasets or not."}
    )
    pad_to_max_length: bool = field(
        default=True,
        metadata={
            "help": (
                "Whether to pad all samples to `max_seq_length`. "
                "If False, will pad the samples dynamically when batching to the maximum length in the batch."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the training data."}
    )
    validation_file: Optional[str] = field(
        default=None, metadata={"help": "A csv or a json file containing the validation data."}
    )
    test_file: Optional[str] = field(default=None, metadata={"help": "A csv or a json file containing the test data."})

    def __post_init__(self):
        if self.task_name is not None:
            self.task_name = self.task_name.lower()
            if self.task_name not in task_to_keys.keys():
                raise ValueError("Unknown task, you should pick one in " + ",".join(task_to_keys.keys()))
        elif self.dataset_name is not None:
            pass
        elif self.train_file is None or self.validation_file is None:
            raise ValueError("Need either a GLUE task, a training/validation file or a dataset name.")
        else:
            train_extension = self.train_file.split(".")[-1]
            assert train_extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            validation_extension = self.validation_file.split(".")[-1]
            assert (
                validation_extension == train_extension
            ), "`validation_file` should have the same extension (csv or json) as `train_file`."


@dataclass
class ModelArguments:
    model_name_or_path: str = field(
        default='distilbert/distilbert-base-uncased',
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizers (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name, or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `huggingface-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )
    ignore_mismatched_sizes: bool = field(
        default=False,
        metadata={"help": "Will enable loading a pretrained model whose head dimensions are different."},
    )
@dataclass
class TrainingArguments(transformers.TrainingArguments):
    output_dir: str = field(
        default="./results",
        metadata={"help": "The output directory where the model predictions and checkpoints will be written."}
    )
    adapter: str = field(
        default=True,
        metadata={"help": "Whether you wanna train adapter or fine-tune"}
    )



parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
filtered_args = filter_unused_args(sys.argv)
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args=filtered_args)

In [6]:
# Set up logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN)
set_seed(42)

In [7]:
### Set Hyperparams Here
model_args.model_name_or_path = 'distilbert/distilbert-base-uncased'
data_args.dataset_name = 'super_glue'
data_args.task_name = 'boolq'
training_args.adapter = True

In [8]:
from adapters import BnConfig, PrefixTuningConfig, MAMConfig, CompacterPlusPlusConfig, UniPELTConfig,\
                    IA3Config, LoRAConfig, PromptTuningConfig,\
                    ConfigUnion

string_to_config = {'seq_bn': BnConfig(mh_adapter=True, output_adapter=True, reduction_factor=16, non_linearity="relu"),
                   "prefix_tuning": PrefixTuningConfig(flat=False, prefix_length=30),
                   "mam_adapter": MAMConfig(),
                   "compacter_plusplus": CompacterPlusPlusConfig(),
                   "unipelt":UniPELTConfig(),
                   "lora": LoRAConfig(r=8, alpha=16),
                   "seq_bn_16_2_relu": ConfigUnion(
                            BnConfig(mh_adapter=True, output_adapter=False, reduction_factor=16, non_linearity="relu"),
                            BnConfig(mh_adapter=False, output_adapter=True, reduction_factor=2, non_linearity="relu"),
                        ),
                    "seq_bn_16_2_tanh": ConfigUnion(
                            BnConfig(mh_adapter=True, output_adapter=False, reduction_factor=16, non_linearity="tanh"),
                            BnConfig(mh_adapter=False, output_adapter=True, reduction_factor=2, non_linearity="tanh"),
                        ),
                    "ia3":IA3Config(),
                    'prompt_tuning': PromptTuningConfig(prompt_length=10)
#                     'loreft':ReftConfig(layers="all", prefix_positions=3, suffix_positions=0, r=1, orthogonality=True),
#                     'noreft':ReftConfig(layers="all", prefix_positions=3, suffix_positions=0, r=1, orthogonality=False),
#                     'direft':ReftConfig(layers="all", prefix_positions=3, suffix_positions=0, r=1, orthogonality=False, subtract_projection=False)
                   }

In [25]:
def preprocess_function(examples):
    args = (
        (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    )
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
    if data_args.task_name is not None:
        result = metric.compute(predictions=preds, references=p.label_ids)
        if len(result) > 1:
            result["combined_score"] = np.mean(list(result.values())).item()
        return result
    elif is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}
    
def preprocess_data(raw_datasets,data_args,model,is_regression,num_labels,label_list,tokenizer):
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        load_from_cache_file=not data_args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )

    train_dataset = raw_datasets["train"]
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))

    eval_dataset = raw_datasets["validation_matched" if data_args.task_name == "mnli" else "validation"]
    if data_args.max_eval_samples is not None:
        max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
        eval_dataset = eval_dataset.select(range(max_eval_samples))

    predict_dataset = raw_datasets["test_matched" if data_args.task_name == "mnli" else "test"]
    if data_args.max_predict_samples is not None:
        max_predict_samples = min(len(predict_dataset), data_args.max_predict_samples)
        predict_dataset = predict_dataset.select(range(max_predict_samples))

    metric = evaluate.load("super_glue", data_args.task_name)


    data_collator = default_data_collator
    
    return train_dataset, eval_dataset, predict_dataset, data_args, data_collator, model, metric

In [26]:
def run_model(model,training_args,train_dataset,eval_dataset,tokenizer,data_collator):

    trainer_class = AdapterTrainer if training_args.adapter else Trainer
    trainer = trainer_class(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    start_time = time.time()

    checkpoint = get_last_checkpoint(training_args.output_dir) if training_args.resume_from_checkpoint is None else training_args.resume_from_checkpoint
    train_result = trainer.train()

    end_time = time.time()
    training_time = end_time - start_time

    metrics = train_result.metrics
    max_train_samples = data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))
    metrics["training_time"] = training_time

    trainer.save_model()  # Saves the tokenizer too for easy upload

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    
    return trainer, train_result

In [36]:
def run_iteration(model_string,data_args,training_args):
    global sentence1_key, sentence2_key,padding,label_to_id,tokenizer,max_seq_length,is_regression,metric
    training_args.adapter_type = model_string
    raw_datasets = load_dataset(
        data_args.dataset_name,
        data_args.task_name,
        cache_dir=model_args.cache_dir,
        use_auth_token=model_args.use_auth_token
    )
    is_regression = data_args.task_name == "stsb"
    if not is_regression:
        label_list = raw_datasets["train"].features["label"].names
        num_labels = len(label_list)
    else:
        num_labels = 1

    config = AutoConfig.from_pretrained(
        model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
        use_auth_token=model_args.use_auth_token,
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        use_auth_token=model_args.use_auth_token
    )
    if training_args.adapter:
        model = AutoAdapterModel.from_pretrained(
            model_args.model_name_or_path,
            from_tf=bool(".ckpt" in model_args.model_name_or_path),
            config=config,
            cache_dir=model_args.cache_dir,
            use_auth_token=model_args.use_auth_token,
            ignore_mismatched_sizes=model_args.ignore_mismatched_sizes
        )
        model.add_classification_head(
        data_args.dataset_name,
        num_labels=num_labels,
        id2label={i: v for i, v in enumerate(label_list)} if not is_regression else None,
        )
        adapter_config_kwargs = {}
        adapter_load_kwargs = {}
        model.add_adapter(training_args.adapter_type, config=string_to_config[training_args.adapter_type])
        if training_args.adapter_type=='prefix_tuning':
            model.eject_prefix_tuning("prefix_tuning")
        model.train_adapter([training_args.adapter_type])
        model.set_active_adapters(training_args.adapter_type)
    else:
        model = transformers.AutoModelForSequenceClassification.from_pretrained(
            model_args.model_name_or_path,
            config=config,
            cache_dir=model_args.cache_dir,
            use_auth_token=model_args.use_auth_token,
            ignore_mismatched_sizes=model_args.ignore_mismatched_sizes
        )
    sentence1_key, sentence2_key = task_to_keys[data_args.task_name]
    padding = "max_length" if data_args.pad_to_max_length else False
    label_to_id = None
    if (
        model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id
        and data_args.task_name is not None
        and not is_regression
    ):
        label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
        if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
            label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
        else:
            logger.warning(
                "Your model seems to have been trained with labels, but they don't match the dataset: ",
                f"model labels: {list(sorted(label_name_to_id.keys()))}, dataset labels: {list(sorted(label_list))}."
                "\nIgnoring the model labels as a result.",
            )
    elif data_args.task_name is None and not is_regression:
        label_to_id = {v: i for i, v in enumerate(label_list)}
    if label_to_id is not None:
        model.config.label2id = label_to_id
        model.config.id2label = {id: label for label, id in config.label2id.items()}
    elif data_args.task_name is not None and not is_regression:
        model.config.label2id = {l: i for i, l in enumerate(label_list)}
        model.config.id2label = {id: label for label, id in config.label2id.items()}

    max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

    train_dataset, eval_dataset, predict_dataset, data_args, data_collator, model, metric = \
                            preprocess_data(raw_datasets,data_args,model,is_regression,num_labels,label_list,tokenizer)
    trainer, train_result = run_model(model,training_args,train_dataset,eval_dataset,tokenizer,data_collator)
    
    eval_metrics = trainer.evaluate(eval_dataset=eval_dataset)
    eval_accuracy = eval_metrics['eval_accuracy']
    
    return eval_accuracy

In [37]:
accuracies = dict()
for model_string in string_to_config.keys():
    accuracies[model_string] = run_iteration(model_string,data_args,training_args)

/home/hice1/maleem3/.local/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprec

Step,Training Loss
500,0.666200
1000,0.650100
1500,0.627900
2000,0.616000
2500,0.614500
3000,0.597000
3500,0.610500


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   902945GF
  train_loss               =     0.6257
  train_runtime            = 0:01:26.65
  train_samples            =       9427
  train_samples_per_second =    326.346
  train_steps_per_second   =     40.815
  training_time            =    86.8561


/home/hice1/maleem3/.local/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprec

Step,Training Loss
500,0.670600
1000,0.655000
1500,0.646600
2000,0.642300
2500,0.651900
3000,0.639200
3500,0.643400


...

Step,Training Loss
500,0.666300
1000,0.654200
1500,0.642100
2000,0.637700
2500,0.643200
3000,0.635500
3500,0.642700


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   885410GF
  train_loss               =     0.6459
  train_runtime            = 0:01:19.44
  train_samples            =       9427
  train_samples_per_second =    355.975
  train_steps_per_second   =     44.521
  training_time            =    79.6342


/home/hice1/maleem3/.local/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hice1/maleem3/.local/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprec

Step,Training Loss
500,0.670800
1000,0.655500
1500,0.649300
2000,0.645900
2500,0.651800
3000,0.643600
3500,0.643200


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   884850GF
  train_loss               =     0.6513
  train_runtime            = 0:01:22.67
  train_samples            =       9427
  train_samples_per_second =    342.084
  train_steps_per_second   =     42.783
  training_time            =      82.86


In [38]:
accuracies

{'seq_bn': 0.6648318042813456,
 'prefix_tuning': 0.6275229357798165,
 'mam_adapter': 0.6688073394495413,
 'compacter_plusplus': 0.635474006116208,
 'unipelt': 0.6685015290519878,
 'lora': 0.65565749235474,
 'seq_bn_16_2_relu': 0.681651376146789,
 'seq_bn_16_2_tanh': 0.6889908256880733,
 'ia3': 0.6370030581039755,
 'prompt_tuning': 0.6238532110091743}

In [44]:
metrics = train_result.metrics
print("Training metrics:", metrics)

eval_metrics = trainer.evaluate(eval_dataset=eval_dataset)
print("Evaluation metrics:", eval_metrics)

if 'eval_accuracy' in eval_metrics:
    accuracy = eval_metrics['eval_accuracy']
    print(f"Accuracy: {accuracy:.4f}")

Training metrics: {'train_runtime': 94.1891, 'train_samples_per_second': 300.258, 'train_steps_per_second': 37.552, 'total_flos': 950701973050368.0, 'train_loss': 0.6461303981644034, 'epoch': 3.0, 'train_samples': 9427, 'training_time': 94.3541829586029}


Evaluation metrics: {'eval_loss': 0.640145480632782, 'eval_accuracy': 0.6379204892966361, 'eval_runtime': 4.9988, 'eval_samples_per_second': 654.162, 'eval_steps_per_second': 81.82, 'epoch': 3.0}
Accuracy: 0.6379


In [14]:
metrics = train_result.metrics
print("Training metrics:", metrics)

eval_metrics = trainer.evaluate(eval_dataset=eval_dataset)
print("Evaluation metrics:", eval_metrics)

if 'eval_accuracy' in eval_metrics:
    accuracy = eval_metrics['accuracy']
    print(f"Accuracy: {accuracy:.4f}")

Training metrics: {'train_runtime': 114.2333, 'train_samples_per_second': 247.572, 'train_steps_per_second': 30.963, 'total_flos': 936577625347584.0, 'train_loss': 0.5112095499294843, 'epoch': 3.0, 'train_samples': 9427, 'training_time': 114.37978649139404}


Evaluation metrics: {'eval_loss': 0.7866023778915405, 'eval_accuracy': 0.7067278287461773, 'eval_runtime': 3.8374, 'eval_samples_per_second': 852.134, 'eval_steps_per_second': 106.582, 'epoch': 3.0}


In [24]:
def test():
    print(a)
def test_():
    global a
    a=5
    test()
test_()

5
